# Mô hình hoạt động của RAG

![Mô hình hoạt động của RAG](415525150_261742853591748_3615540183803465017_n.jpg)


# Cài thư viện cần thiết

In [ ]:
%pip install openai langchain numpy pandas faiss-cpu PyPDF2

# Import các thư viện cần thiết

In [3]:
from openai import OpenAI
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter # Chia nhỏ văn bản thành các đoạn
from langchain.schema import Document # Kiểu dữ liệu 
import faiss # Cơ sở dữ liệu
import numpy as np
import pandas as pd
import os
# import json


client = OpenAI(
    base_url="http://26.203.51.178:5555/v1",
    api_key="dont_need"
)

# Đọc file pdf

In [ ]:
def openPdfFile(pdf_path):
    documents = []
    pdf_reader = PdfReader(pdf_path)
    
    for page_num, page in enumerate(pdf_reader.pages):
        page_text = page.extract_text()
        document = Document(
              page_content=page_text,
              metadata= {
                    "source": pdf_path,
                    "page": page_num,
                    "content": ""
              }
        )
        documents.append(document)
    return documents

In [5]:
print(openPdfFile('datasets/Machinelearningyearning-Vi.pdf'))

[Document(metadata={'source': 'datasets/Machinelearningyearning-Vi.pdf', 'page': 0}, page_content='ANDREW NG\nKhátKhaoHọcMáy\nMachineLearningYearning\nChiến lược kỹ thuật cho các kỹ sư AI\ntrong kỷ nguyên Học Sâu\nNhómDịchThuật\nMachineLearningCơBản'), Document(metadata={'source': 'datasets/Machinelearningyearning-Vi.pdf', 'page': 1}, page_content='MỤC LỤC\nGiới thiệu\n1. Tại sao cần chiến lược Học Máy\n2. Cách sử dụng cuốn sách khi làm việc nhóm\n3. Kiến thức tiền đề và Ký hiệu\n4. Quy mô là động lực phát triển học máy\nPhần 1: Chuẩn bị tập phát triển và tập kiểm tra\n5. Tập phát triển và tập kiểm tra\n6. Tập phát triển và tập kiểm tra nên có cùng phân phối\n7. Tập phát triển/kiểm tra cần lớn đến mức nào?\n8. Thiết lập một phép đo đơn trị làm mục tiêu tối ư u\n9. Phép đo để tối ưu và phép đo thỏa mãn\n10. Xây dựng một tập phát triển và một phép đo sẽ t ăng tốc quá trình làm\nviệc\n11. Khi nào cần thay đổi tập phát triển/kiểm tra và  các phép đo\n12. Điều cần nhớ: Thiết lập các tập phá

# Chia nhỏ ra thành từng đoạn

In [7]:
def text_split(docs):
    text_splitter = CharacterTextSplitter(
        chunk_size=512, # Tách 512 độ dài 
        chunk_overlap=100, # Lấy 100 độ dài ở đằng trước
        separator="\n", # Tách theo dòng
        length_function=len # hàm tính độ dài. dùng len() để tính theo kí tự
    )
    texts = text_splitter.split_documents(docs)
    return texts

# Hàm Embedding Text

In [9]:
def embeddingText(text):
    response = client.embeddings.create(
        model="text-embedding-nomic-embed-text-v1.5",
        input=text,
        encoding_format="float",
        
    )

    return response.data[0].embedding

# Đọc dữ liệu từ các file và thêm vào cơ sở dữ liệu

* ### Bước 1: Lấy tên các file pdf đang có ở trong folder
* ### Bước 2: Đọc file pdf và chia nhỏ thành từng đoạn
* ### Bước 3: Embedding
* ### Bước 4: Thêm vào cơ sở dữ liệu

In [10]:
# Bước 1: 

# Tên thư mục cần kiểm tra
folder_name = 'datasets'
faiss_name = "faiss_index.index"
# Khởi tạo danh sách để chứa các file PDF
pdf_files = []

# Kiểm tra xem thư mục có tồn tại không
if os.path.exists(folder_name) and os.path.isdir(folder_name):
    # Duyệt qua tất cả các file trong thư mục
    for file in os.listdir(folder_name):
        # Kiểm tra nếu file có phần mở rộng là .pdf
        if file.lower().endswith('.pdf'):
            pdf_files.append(f"{folder_name}/{file}")

# Bước 2:

documents = []
for pdf_file in pdf_files:
    document = text_split(openPdfFile(pdf_file))
    documents = documents + document

# for document in documents:
#     print(document.page_content)

# Bước 3:
embedding_vectors = [embeddingText(doc.page_content) for doc in documents]
embedding_vectors_np = np.array(embedding_vectors).astype(np.float32)

# Bước 4:

dim = embedding_vectors_np.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embedding_vectors_np)
faiss.write_index(index, faiss_name)



# Tìm kiếm trong cơ sở dữ liệu

In [11]:
def faiss_query(question, k=7):
    index = faiss.read_index(faiss_name)
    query_embedding =  embeddingText(question)
    query_embedding_np = np.array([query_embedding]).astype(np.float32)
    _, indices = index.search(query_embedding_np, k)
    contexts = [documents[i] for i in indices[0]]
    content = ""
    for context in contexts:
        content+=context.page_content + "\n\n"
    return content

# LLM Tạo câu trả lời

In [12]:
def chat_with_gemini(messages):
    response = client.chat.completions.create(
        model="qwen3-0.6b",
        messages=messages,
        temperature=0.3,
        
    )
    return response.choices[0].message.content

In [13]:
messages = [
    {
        "role": "user",
        'content': "Giới thiệu về học tăng cường trong AI"
    }
]

question = messages[-1]['content']
content = faiss_query(question,7)

system_prompt = {
    "role": "system",
    "content": f"""Bạn là một trợ lí ảo của Nguyễn Ngọc An. Tên của bạn là 13Bee.
    Dựa vào những kiến thức dưới đây để trả lời:
    '{content}'
"""
}


messages = [system_prompt] + messages
# print(messages)
# print(chat_with_gemini(messages))

In [14]:
content

'độ cao có nhiều biến động đối với hành khách không,  cùng nhiều yếu tố khác. Thiết\nkế hàm điểm thưởng tốt là một việc không hề dễ dàng .\nVới một hàm điểm thưởng R(T) cho trước, công việc của thuật toán học tăng cường\nlà điều khiển trực thăng sao cho nó đạt được điểm t hưởng cao nhất max R(T). Tuy\nnhiên, thuật toán học tăng cường có nhiều phép xấp xỉ và có thể sẽ không thànhT\n82\n\nHệ âm vị, một phát kiến của ngành ngôn ngữ học, là một biểu diễn không hoàn\nhảo của âm thanh thoại. Trong những trường hợp mà h ệ âm vị là một xấp xỉ kém\ncủa thực tế, áp đặt một thuật toán sử dụng biểu diễ n âm vị sẽ giới hạn chất lượng\ncủa hệ thống tiếng nói.\nCác thành phần được thiết kế thủ công này giới hạn chất lượng tiềm năng của hệ\nthống tiếng nói. Tuy nhiên, việc sử dụng các thành phần được thiết kế thủ công\ncũng có một vài ưu điểm:\n\nthể giúp giải quyết các vấn đề về phương sai, nhưng  nó cũng có thể làm tăng độ\nchệch. Việc giảm một ít số lượng các đặc trưng (giả  sử từ 1.000 xuống 900 đ

In [15]:
print(chat_with_gemini(messages))

<think>
Okay, the user wants an introduction to reinforcement learning in AI. Let me start by recalling what I know from the previous information.

First, the main points from the provided text: Reinforcement learning (RL) is about training agents to make decisions that maximize cumulative rewards. It's different from supervised and unsupervised methods because it learns through interaction with an environment. The text mentions that RL has a reward function and aims to maximize R(T), which is the total reward.

Then there's something about the complexity of RL algorithms, like overfitting, which might be relevant here. Also, the mention of the "hệ âm vị" (audio version) as a method for improving systems when it doesn't fully represent real-world data. But I need to check if that's part of the introduction or just additional info.

The user also provided some examples from previous interactions where they mentioned challenges with data consistency and different training distributions. 